# Sequential Testing

'Peeking' is a dangerous source of inflated false positives in an A/B test. However, the desire to 'know results early'
is not always a bad one -
for example, when a significant loss may be incurred by delaying an experiment decision,
such as launching a new feature ahead of a major event ([More Examples Here](https://docs.statsig.com/experiments-plus/sequential-testing)).
Sequential Testing is the body of work that develops and recommends
methods in which A/B Test practicioners can, under controlled settings, 'peak' and make earlier decisions as necessary.

A nice review of the current state of the literature of Sequential Stopping can be found at
([Larsen et al., 2023](https://arxiv.org/pdf/2212.11366.pdf)). There is also a nice high level overview by Spotify
([Schultzberg and Ankargren, 2023](https://engineering.atspotify.com/2023/03/choosing-sequential-testing-framework-comparisons-and-discussions/)).
Note that the Spotify article's comment on StatSig's implementation is outdated - like many other companies, StatSig has switched to the mSPRT
methodology described in ([Zhao et al., 2019](https://arxiv.org/pdf/1905.10493.pdf)).
More on StatSig's implementation at ([Stewart, 2023](https://www.statsig.com/blog/sequential-testing-on-statsig))

In this Simulation, we will code out sequential testing and compare it with a peeking test in the following areas:
1. FPR (False Positive Rate)
2. Power

for 3 different kinds of tests:
1. fixed horizon test
2. peeking test
3. sequential test (mSPRT)

These are heavily inspired by ([Stewart, 2023](https://www.statsig.com/blog/sequential-testing-on-statsig)) and ([Li, 2022](https://towardsdatascience.com/wish-tackles-peeking-with-always-valid-p-values-8a0782ac9654))

In [1]:
import numpy as np
import pandas as pd
import uuid
from scipy.stats import ttest_ind
import scipy.stats as stats

# Data Generation

In [132]:
# generate data
def sample_data(traffic_per_day=100, days=14, param=0.037):
  """
  generate A/B test data
  """
  # Generate Poisson
  samples = np.random.binomial(n=1, p=param, size=(traffic_per_day * days))

  # Generate UUIDs
  uuids = [str(uuid.uuid4()) for _ in range(days * traffic_per_day)]

  # Generate evenly spaced days
  days_column = np.repeat(np.arange(1, days + 1), traffic_per_day)

  # Combine UUIDs, days, and numpy array
  df = pd.DataFrame({
      'userid': uuids,
      'day': days_column,
      'action_count': samples
  })


  return df

# generate standard control and treatment
def test_data(relative_lift=0.1, traffic_per_day=100, days=14, param=0.037):
  """
  Control and Treatment
  """
  return sample_data(traffic_per_day, days, param), sample_data(traffic_per_day, days, param*(1+relative_lift))

# Standard Fixed Horizon Test

In [3]:
def fixed_horizon_test(control, treatment, alpha=0.05):
  """
  performs ttest once using full data
  """
  t_statistic, p_value = ttest_ind(treatment['action_count'], control['action_count'], equal_var=False)

  return 1 if p_value < alpha else 0

# Peeking Test

In [4]:
def peeking_test(control, treatment, alpha=0.05):
    """
    Perform a peeking test using Welch's t-test.
    """
    # Iterate over each day
    for day in range(1, max(control['day']) + 1):
        # Filter data up to the current day
        control_data = control[control['day'] <= day]['action_count']
        treatment_data = treatment[treatment['day'] <= day]['action_count']

        # Perform Welch's t-test
        t_statistic, p_value = ttest_ind(control_data, treatment_data, equal_var=False)

        # Check if p-value is significant
        if p_value < alpha:
            return 1  # Significant result found, return 1

    # No significant result found after testing all days
    return 0

# Sequential Test (mSPRT)

In [51]:
def msprt_pval(control, treatment, alpha = 0.05):
  """
  performs msprt
  """
  # Iterate over each day
  for day in range(1, max(control['day']) + 1):
      # Filter data up to the current day
      x_c = control[control['day'] <= day]['action_count']
      x_t = treatment[treatment['day'] <= day]['action_count']

      # variance of the difference in means
      v = (np.var(x_c) / len(x_c)) + (np.var(x_t) / len(x_t))

      # Z-score corresponding to 1 - alpha/2
      Z_alpha = stats.norm.ppf(1-alpha/2)

      # mixing parameter
      t = (Z_alpha**2) * ((np.var(x_t) + np.var(x_c)) / (len(x_c) + len(x_t)))

      # e-term
      eterm = (((len(x_c)+len(x_t))**2) * (t**2) * ((np.mean(x_t) - np.mean(x_c))**2)) / (2*v*(v+(len(x_t)+len(x_c))*(t**2)))

      delta = np.sqrt(v/(v+(len(x_c)+len(x_t))*(t**2)))*np.exp(eterm)

      # p-val
      p_value = 1/delta

      # Check if p-value is significant
      if p_value < alpha:
          return 1  # Significant result found, return 1

  # No significant result found after testing all days
  return 0


In [5]:
def msprt_ci(control, treatment, alpha = 0.05):
  """
  performs msprt
  """
  # Iterate over each day
  for day in range(1, max(control['day']) + 1):
      # Filter data up to the current day
      x_c = control[control['day'] <= day]['action_count']
      x_t = treatment[treatment['day'] <= day]['action_count']

      # variance of the difference in means
      v = (np.var(x_c) / len(x_c)) + (np.var(x_t) / len(x_t))

      # Z-score corresponding to 1 - alpha/2
      Z_alpha = stats.norm.ppf(1-alpha/2)

      # mixing parameter
      t = (Z_alpha**2) * ((np.var(x_t) + np.var(x_c)) / (len(x_c) + len(x_t)))

      # marin of error term
      me = np.sqrt(((v*(v+t))/t) * (-2*np.log(alpha/2) - np.log(v/(v+t))))

      # sample mean
      e = np.mean(x_t) - np.mean(x_c)

      # ci
      ci_l = e - me
      ci_u = e + me

      # Check if significant
      if ci_u < 0 or ci_l > 0:
          return 1  # Significant result found, return 1

  # No significant result found after testing all days
  return 0


# Simulate A/A Tests

Let's run a 1000 A/A tests each for each method:

### Fixed Horizon Test

In [142]:
results = []
for i in range(1000):
  control, treatment = test_data(relative_lift = 0)
  results.append(fixed_horizon_test(control=control, treatment=treatment))

print('FPR: ',np.mean(results))

FPR:  0.051


### Peeking Test

In [143]:
results = []
for i in range(1000):
  control, treatment = test_data(relative_lift = 0)
  results.append(peeking_test(control=control, treatment=treatment))

print('FPR: ',np.mean(results))

FPR:  0.215


### Sequential Test (mSPRT)

In [144]:
results = []
for i in range(1000):
  control, treatment = test_data(relative_lift = 0)
  results.append(msprt_ci(control=control, treatment=treatment))

print('FPR: ',np.mean(results))

FPR:  0.002


We see that these results are very similar to StatSig's results at ([Stewart, 2023](https://www.statsig.com/blog/sequential-testing-on-statsig)).

# Power

First, we use van Belle's formula to get a sample size estimate for 80% power and 5% alpha:

$$n = \frac{16\sigma^2}{\delta^2}$$

for each variant

In [129]:
def sample_size_calculator(relative_lift=0.01):
  control, treatment = test_data(relative_lift=relative_lift, traffic_per_day=100, days=14, param=0.037)

  n = (16 * (np.var(control['action_count']))) / ((np.mean(control['action_count']*(relative_lift)))**2)

  return round(n)

In [130]:
sample_size_calculator(relative_lift = 0.1)

37698

We need about 40,000 per variant. So we can do a 100 day test with 400 traffic per day.

### Fixed Horizon Test

In [138]:
results = []
for i in range(100):
  control, treatment = test_data(relative_lift = 0.1,days=100,traffic_per_day = 400)
  results.append(fixed_horizon_test(control=control, treatment=treatment))

print('Power: ',np.mean(results))

Power:  0.75


In [140]:
results = []
for i in range(100):
  control, treatment = test_data(relative_lift = 0.1,days=100, traffic_per_day = 400)
  results.append(peeking_test(control=control, treatment=treatment))

print('Power: ',np.mean(results))

Power:  0.87


In [141]:
results = []
for i in range(100):
  control, treatment = test_data(relative_lift = 0.1,days=100, traffic_per_day = 400)
  results.append(msprt_ci(control=control, treatment=treatment))

print('Power: ',np.mean(results))

Power:  0.32


Much like in ([Stewart, 2023](https://www.statsig.com/blog/sequential-testing-on-statsig)), the Power is lower in mSPRT than it is with fixed horizon. This is the primary reason for why Sequential Testing is not told to be unanimously superior. If it were as superior, then sequential testing would always be recommended over fixed horizon.